In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("Research essay A0284714Y.pdf")

documents = loader.load()


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 43 0 (offset 0)


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitting = RecursiveCharacterTextSplitter(
    chunk_size = 600, chunk_overlap = 0, 
    length_function = len, is_separator_regex = False
)

chunks = textSplitting.split_documents(documents)

print(len(chunks))

33


In [5]:
from sentence_transformers import SentenceTransformer
import numpy

max= 4096

embedding_model = SentenceTransformer('Snowflake/snowflake-arctic-embed-l', trust_remote_code=True)

chunk_texts = [chunk.page_content for chunk in chunks]

embeddings = embedding_model.encode(chunk_texts, show_progress_bar=True)

print(embeddings)

embeddings_np = numpy.array(embeddings).astype('float32') #fass takes in 



You try to use a model that was created with version 2.7.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



Batches: 100%|██████████| 2/2 [00:31<00:00, 15.54s/it]

[[ 0.00747334 -0.08902401  0.02077446 ... -0.01880907 -0.04058751
  -0.00896564]
 [ 0.01578885 -0.03381775  0.01899354 ...  0.00736156  0.02683321
   0.00205371]
 [ 0.00219827 -0.05417677  0.01220722 ... -0.0225475  -0.00642097
  -0.01477658]
 ...
 [ 0.01143977 -0.08681262 -0.00466331 ... -0.01262989  0.00927003
  -0.02335927]
 [ 0.00517207 -0.04478921 -0.00024162 ... -0.03591544 -0.01128993
  -0.05130975]
 [-0.03160992 -0.07529064  0.00959103 ...  0.05392865 -0.01141339
   0.00206803]]


In [6]:

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("Salesforce/SFR-Embedding-2_R")

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

# Each query must come with a one-sentence instruction that describes the task
task = 'Given a web search query, retrieve relevant passages that answer the query'
queries = [
    get_detailed_instruct(task, 'How to bake a chocolate cake'),
    get_detailed_instruct(task, 'Symptoms of the flu')
]
# No need to add instruction for retrieval documents
passages = [
    "To bake a delicious chocolate cake, you'll need the following ingredients: all-purpose flour, sugar, cocoa powder, baking powder, baking soda, salt, eggs, milk, vegetable oil, and vanilla extract. Start by preheating your oven to 350°F (175°C). In a mixing bowl, combine the dry ingredients (flour, sugar, cocoa powder, baking powder, baking soda, and salt). In a separate bowl, whisk together the wet ingredients (eggs, milk, vegetable oil, and vanilla extract). Gradually add the wet mixture to the dry ingredients, stirring until well combined. Pour the batter into a greased cake pan and bake for 30-35 minutes. Let it cool before frosting with your favorite chocolate frosting. Enjoy your homemade chocolate cake!",
    "The flu, or influenza, is an illness caused by influenza viruses. Common symptoms of the flu include a high fever, chills, cough, sore throat, runny or stuffy nose, body aches, headache, fatigue, and sometimes nausea and vomiting. These symptoms can come on suddenly and are usually more severe than the common cold. It's important to get plenty of rest, stay hydrated, and consult a healthcare professional if you suspect you have the flu. In some cases, antiviral medications can help alleviate symptoms and reduce the duration of the illness."
]

embeddings = model.encode(queries + passages)
scores = util.cos_sim(embeddings[:2], embeddings[2:]) * 100
print(scores.tolist())
# [[40.132083892822266, 25.032529830932617], [15.006855010986328, 39.93733215332031]]



No sentence-transformers model found with name Salesforce/SFR-Embedding-2_R. Creating a new one with MEAN pooling.
c:\Users\joshi\Desktop\KAYO\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


KeyboardInterrupt: 

In [8]:
import faiss #facebook ai similarity search 

query = "tourism in india"

query_embedding = embedding_model.encode(query, show_progress_bar=True)

query_embedding_np = numpy.array([query_embedding]).astype('float32')

nearest_neighbours = 5

dimension = embeddings_np.shape[1]

index = faiss.IndexFlatL2(dimension)
 
index.add(embeddings_np)

distance, indices = index.search(query_embedding_np, nearest_neighbours)

#The lower the distance, the more similar the chunk is to your query.
#hese are the positions of the most similar chunks within your original chunks list

print("Distance: ", distance)


print("Indices: ", indices)

similarity_scores = [ 1 / (1 + dist) for dist in distance[0] ]

print("Similarity scores: ", similarity_scores)


Batches: 100%|██████████| 1/1 [00:00<00:00,  9.46it/s]

Distance:  [[0.48538578 0.49017334 0.4978583  0.5116787  0.52909636]]
Indices:  [[24 27 25 13  3]]
Similarity scores:  [0.6732257817797441, 0.6710628711857465, 0.6676199006346705, 0.6615162354663083, 0.6539810197092225]


In [10]:
#Retrieving the chunks using distance and indices

for i in range(len(indices)) :
    for rank in range(nearest_neighbours):
        chunk_index = indices[i][rank]
        similarity = similarity_scores[rank]
        chunk_text = chunks[chunk_index]
        print(f"Rank {rank+1}: {chunk_text.page_content} with similarity {similarity}")
        

Rank 1: showcasing the diverse offerings of India create positive content and change the perception of people.   Furthermore, there is immense potential for further growth and collaboration in tourism promotion between Singapore and India. This could involve joint marketing campaigns, cultural exchange programs, and facilitation of business partnerships in the tourism sector. Policymakers, tourism agencies, and stakeholders should prioritize initiatives that leverage the strengths of both countries and address existing challenges.  Conclusion: with similarity 0.6732257817797441
Rank 2: India looks promising, with immense potential for joint marketing campaigns, cultural exchange programs, and facilitation of business partnerships in the sector. with similarity 0.6710628711857465
Rank 3: Tourism has played a pivotal role in strengthening the multifaceted relationship between India and Singapore. The thriving tourism industry has fostered political, economic, and cultural ties between th

In [12]:
#LLM 
import os
import google.generativeai as genai

api_key = os.getenv("GENAI_API_KEY")
if not api_key:
    raise ValueError("API key not found in environment variables.")

genai.configure(api_key=api_key)

llm = genai.GenerativeModel('gemini-1.5-flash')


In [13]:
prompt = " Please answer the following question in context of the provided text:\n"
prompt += "Question: " + query + "\n"
for i in range(len(indices)) :
    for rank in range(nearest_neighbours):
        chunk_index = indices[i][rank]
        similarity = similarity_scores[rank]
        chunk_text = chunks[chunk_index]
        prompt += "Context: " + chunk_text.page_content + "\n"
        
response = llm.generate_content(prompt)

print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The provided text focuses on the **positive impact of tourism on the relationship between Singapore and India**, rather than providing a detailed analysis of tourism in India itself.  However, we can glean some insights:\n\n* **India's tourism offerings are diverse and hold immense potential:** The text highlights that India's offerings are diverse and capable of changing perceptions. \n* **India is a popular destination for Singaporean tourists:** The text mentions that India's \"splendour\" draws \"a substantial number of Singaporean visitors each year.\"\n* **Collaboration in tourism promotion between Singapore and India:** There is significant potential for joint marketing campaigns, cultural exchange programs, and business partnerships to further boost 